In [98]:
!mkdir -p ~/agave/aaron

%cd ~/agave/aaron

!pip3 install --upgrade setvar

import re
import os
import sys
from setvar import *

!auth-tokens-refresh

/home/jovyan/agave/aaron
Requirement already up-to-date: setvar in /opt/conda/lib/python3.6/site-packages
You are using pip version 9.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
Token for sandbox:AaronHolt successfully refreshed and cached for 14400 seconds
c32c3133b4f486ad31714231026f94


In [99]:
#Empty Deploy Path
os.environ["AGAVE_STORAGE_SYSTEM_ID"] = "sandbox-storage-AaronHolt"
#!files-mkdir -S ${AGAVE_STORAGE_SYSTEM_ID} -N /home/jovyan/funwave_deploy

In [100]:
!files-mkdir -S ${AGAVE_STORAGE_SYSTEM_ID} -N /home/jovyan/FUNWAVE-TVD/build

Successfully created folder /home/jovyan/FUNWAVE-TVD/build


In [101]:
!ssh sandbox "cd ~/FUNWAVE-TVD && git checkout -b dev"
writefile("version.txt","""3
2
0
dev""")
#!files-upload -S ${AGAVE_STORAGE_SYSTEM_ID} -F version.txt /home/jovyan/funwave_deploy
!files-upload -S ${AGAVE_STORAGE_SYSTEM_ID} -F version.txt /home/jovyan/FUNWAVE-TVD/build

fatal: A branch named 'dev' already exists.
Writing file `version.txt'
Uploading version.txt...
######################################################################## 100.0%


Build wrapper now clones FWV too

In [102]:
writefile("funwave-build-wrapper.txt","""

VERSION=$(cat version.txt | paste -sd "." -)

git clone /home/jovyan/FUNWAVE-TVD
cd FUNWAVE-TVD
git checkout -b dev

sudo docker build \
    --build-arg "BUILD_DATE=\${AGAVE_JOB_SUBMIT_TIME}" \
    --build-arg "VERSION=\${VERSION}" \
    --rm -t funwave-tvd:\${VERSION} .

docker inspect funwave-tvd:\${VERSION}

""")
#!files-upload -S ${AGAVE_STORAGE_SYSTEM_ID} -F funwave-build-wrapper.txt /home/jovyan/funwave_deploy
!files-upload -S ${AGAVE_STORAGE_SYSTEM_ID} -F funwave-build-wrapper.txt /home/jovyan/FUNWAVE-TVD/build

Writing file `funwave-build-wrapper.txt'
Uploading funwave-build-wrapper.txt...
######################################################################## 100.0%


In [103]:
writefile("Dockerfile","""
FROM stevenrbrandt/science-base
MAINTAINER Steven R. Brandt <sbrandt@cct.lsu.edu>

ARG BUILD_DATE
ARG VERSION


LABEL org.agaveplatform.ax.architecture="x86_64"                                \
      org.agaveplatform.ax.build-date="\$BUILD_DATE"                             \
      org.agaveplatform.ax.version="\$VERSION"                             \
      org.agaveplatform.ax.name="${AGAVE_USERNAME}/funwave-tvd"    \
      org.agaveplatform.ax.summary="Funwave-TVD is a code to simulate the shallow water and Boussinesq equations written by Dr. Fengyan Shi." \
      org.agaveplatform.ax.vcs-type="git"                                       \
      org.agaveplatform.ax.vcs-url="https://github.com/fengyanshi/FUNWAVE-TVD" \
      org.agaveplatform.ax.license="BSD 3-clause"
      
USER root
RUN mkdir -p /home/install/FUNWAVE-TVD
RUN chown jovyan /home/install/FUNWAVE-TVD
RUN pwd
RUN ls
COPY --chown=jovyan:jovyan ./ /home/install/FUNWAVE-TVD

USER jovyan

WORKDIR /home/install/FUNWAVE-TVD/src
RUN perl -p -i -e 's/FLAG_8 = -DCOUPLING/#$&/' Makefile && \
    make

RUN mkdir -p /home/jovyan/rundir
WORKDIR /home/jovyan/rundir
""")
!files-upload -S ${AGAVE_STORAGE_SYSTEM_ID} -F Dockerfile /home/jovyan/FUNWAVE-TVD

Writing file `Dockerfile'
Uploading Dockerfile...
######################################################################## 100.0%


In [104]:
writefile("funwave-build-app.txt","""
{  
   "name":"${AGAVE_USERNAME}-${MACHINE_NAME}-funwave-dbuild",
   "version":"1.0",
   "label":"Builds the funwave docker image",
   "shortDescription":"Funwave docker build",
   "longDescription":"",
   "deploymentSystem":"${AGAVE_STORAGE_SYSTEM_ID}",
   "deploymentPath":"FUNWAVE-TVD/build/",
   "templatePath":"funwave-build-wrapper.txt",
   "testPath":"version.txt",
   "executionSystem":"${AGAVE_EXECUTION_SYSTEM_ID}",
   "executionType":"CLI",
   "parallelism":"SERIAL",
   "modules":[],
   "inputs":[],
   "parameters":[{
     "id" : "code_version",
     "value" : {
       "visible":true,
       "required":true,
       "type":"string",
       "order":0,
       "enquote":false,
       "default":"latest"
     },
     "details":{
         "label": "Version of the code",
         "description": "If true, output will be packed and compressed",
         "argument": null,
         "showArgument": false,
         "repeatArgument": false
     },
     "semantics":{
         "argument": null,
         "showArgument": false,
         "repeatArgument": false
     }
   }],
   "outputs":[]
}
""")
!files-upload -S ${AGAVE_STORAGE_SYSTEM_ID} -F funwave-build-app.txt /home/jovyan/FUNWAVE-TVD/build
#!files-upload -S ${AGAVE_STORAGE_SYSTEM_ID} -F funwave-build-app.txt /home/jovyan/funwave_deploy

Writing file `funwave-build-app.txt'
Uploading funwave-build-app.txt...
######################################################################## 100.0%


In [105]:
!pwd
!ls

/home/jovyan/agave/aaron
Dockerfile		funwave-build.out
funwave-build-app.txt	funwave-build-wrapper.txt
funwave-build.err	job-8913959023177699816-242ac114-0001-007
funwave-build-job.json	version.txt


In [106]:
!ssh sandbox "set -x && cd ~/FUNWAVE-TVD && git add build && git add Dockerfile"
!ssh sandbox "cd ~/FUNWAVE-TVD && git commit -m 'Added Dockerfile and Agave build files.'"

+ cd /home/jovyan/FUNWAVE-TVD
+ git add build
+ git add Dockerfile
On branch dev
nothing to commit, working directory clean


In [107]:
!apps-addupdate -F funwave-build-app.txt

Successfully added app AaronHolt-sandbox-funwave-dbuild-1.0


In [108]:
requestbin_url = !requestbin-create
os.environ['REQUESTBIN_URL'] = requestbin_url[0]
setvar("""
WEBHOOK_URL=${REQUESTBIN_URL}
""")

WEBHOOK_URL=https://requestbin.agaveapi.co/1jp98cj1


In [109]:
!pwd
writefile("funwave-build-job.json","""
 {
   "name":"funwave-build",
   "appId": "${AGAVE_USERNAME}-${MACHINE_NAME}-funwave-dbuild-1.0",
   "maxRunTime":"00:10:00",
   "archive": false,
   "notifications": [
    {
      "url":"${WEBHOOK_URL}",
      "event":"*",
      "persistent":"true"
    }
   ],
   "parameters": {
     "code_version":"latest"
   }
 }
""")

/home/jovyan/agave/aaron
Writing file `funwave-build-job.json'


In [110]:
setvar("""
# Capture the output of the job submit command
OUTPUT=$(jobs-submit -F funwave-build-job.json)
# Parse out the job id from the output
JOB_ID=$(echo $OUTPUT | cut -d' ' -f4)
""")

OUTPUT=Successfully submitted job 517582389412303336-242ac114-0001-007
JOB_ID=517582389412303336-242ac114-0001-007


In [111]:
from time import sleep
for iter in range(25):
    setvar("STAT=$(jobs-status $JOB_ID)")
    stat = os.environ["STAT"]
    sleep(5.0)
    if stat == "FINISHED" or stat == "FAILED":
        break

STAT=STAGED
STAT=SUBMITTING
STAT=SUBMITTING
STAT=SUBMITTING
STAT=RUNNING
STAT=RUNNING
STAT=RUNNING
STAT=RUNNING
STAT=RUNNING
STAT=RUNNING
STAT=RUNNING
STAT=FINISHED


In [112]:
!jobs-output-list ${JOB_ID}

.agave.archive
.agave.log
funwave-build-app.txt
funwave-build-wrapper.txt
funwave-build.err
funwave-build.ipcexe
funwave-build.out
funwave-build.pid
FUNWAVE-TVD
version.txt


In [113]:
!jobs-output-get -P ${JOB_ID} funwave-build.out


Step 1/16 : FROM stevenrbrandt/science-base
 ---> d16f0aff4390
Step 2/16 : MAINTAINER Steven R. Brandt <sbrandt@cct.lsu.edu>
 ---> Using cache
 ---> 73b8632a8acd
Step 3/16 : ARG BUILD_DATE
 ---> Using cache
 ---> 5ea9d4936abd
Step 4/16 : ARG VERSION
 ---> Using cache
 ---> ee968cc63967
Step 5/16 : LABEL org.agaveplatform.ax.architecture="x86_64"                                      org.agaveplatform.ax.build-date="$BUILD_DATE"                                   org.agaveplatform.ax.version="$VERSION"                                   org.agaveplatform.ax.name="AaronHolt/funwave-tvd"          org.agaveplatform.ax.summary="Funwave-TVD is a code to simulate the shallow water and Boussinesq equations written by Dr. Fengyan Shi."       org.agaveplatform.ax.vcs-type="git"                                             org.agaveplatform.ax.vcs-url="https://github.com/fengyanshi/FUNWAVE-TVD"       org.agaveplatform.ax.license="BSD 3-clause"
 ---> Running in 1b0d20bcdb03
Removing intermediate cont

In [114]:
!jobs-output-get -P ${JOB_ID} funwave-build.err

Cloning into 'FUNWAVE-TVD'...
done.
fatal: A branch named 'dev' already exists.
Got permission denied while trying to connect to the Docker daemon socket at unix:///var/run/docker.sock: Get http://%2Fvar%2Frun%2Fdocker.sock/v1.38/containers/funwave-tvd:3.2.0.dev/json: dial unix /var/run/docker.sock: connect: permission denied
